In [1]:
with open("input17.txt","r") as f:
    data = f.read().strip()
data = data.split("\n")

In [2]:
import re
import numpy as np

In [3]:
def process_lines(data):

    lines = []
    xvals = []
    yvals = []
    for line in data:
        vals = re.findall("x|y|\d+",line)
        xI= vals.index("x")
        yI = vals.index("y")
        if xI<yI:
            obj = {"x":list(map(int,vals[xI+1:yI])),
                   "y":list(map(int,vals[yI+1:]))}
        else:
            obj = {"y":list(map(int,vals[yI+1:xI])),
                   "x":list(map(int,vals[xI+1:]))}
        xvals += [x for x in obj["x"]]
        yvals += [x for x in obj["y"]]
        lines.append(obj)
        
    minx, maxx = min(xvals), max(xvals)
    miny, maxy = min(yvals), max(yvals)
    
    return lines, minx, maxx, miny, maxy


In [4]:
def make_array(lines, yrange, xrange, minx, miny):
    arr = np.zeros((yrange+2,xrange+3))
    
    for dct in lines:
        xr = [x-minx+1 for x in dct["x"]]
        yr = [y-miny+1 for y in dct["y"]]

        assert (len(xr)==2 or len(yr)==2)

        if len(xr)==1:
            arr[yr[0]:yr[1]+1,xr[0]] = 1
        else:
            arr[yr[0],xr[0]:xr[1]+1] = 1
    return arr

In [5]:
def get_data(data):
    
    lines, minx, maxx, miny, maxy = process_lines(data)
    xrange = maxx - minx
    yrange = maxy - miny
    
    arr = make_array(lines, yrange, xrange, minx, miny)
    
    return arr, 500-minx

In [11]:
def pwrite(arr):
    wf = []
    for row in arr:
        #print(len(row))
        pprint = ""
        for val in row:
            if val==0:
                pprint += "."
            elif val==1:
                pprint += "#"
            elif val==2:
                pprint += "~"
            elif val==3:
                pprint += "|"
        wf.append(pprint)
    with open("output17.txt","w") as f:
        f.write("\n".join(wf))

In [12]:
def pprint(arr):
    for row in arr:
        pprint = ""
        for val in row:
            if val==0:
                pprint += "."
            elif val==1:
                pprint += "#"
            elif val==2:
                pprint += "~"
            elif val==3:
                pprint += "|"
        print(pprint)


In [7]:
def drip_down(arr,r,c):
    """
    Takes in r,c of | over .
    Drip down until we hit bottom of array, #, or ~
    """
    while r+1<arr.shape[0] and arr[r+1,c]==0:
        r+=1
        arr[r,c]=3
    return arr

In [8]:
def drip_across(arr,r,c_):
    """
    Takes in r,c of | over # or ~
    Drip across until we hit a solid, or are over .
    After dripping across, check to see if we have a solid
    """
    dripped = False
    
    cl = c_
    #left
    while cl>0 and arr[r,cl-1] in [0,3] and arr[r+1,cl] in [1,2]:
        cl-=1
        if arr[r,cl] != 3:
            arr[r,cl] = 3
            dripped = True
    left_wall = arr[r,cl-1]==1
    
    cr = c_
    #right
    while cr<arr.shape[1] and arr[r,cr+1] in [0,3] and arr[r+1,cr] in [1,2]:
        cr+=1
        if arr[r,cr] != 3:
            arr[r,cr] = 3
            dripped = True
    right_wall = arr[r,cr+1]==1
        
    #If bookended by walls, change to "solid"
    if left_wall and right_wall:
        arr[r,cl:cr+1]=2
        dripped = True
        
    return arr, dripped
    

In [9]:
arr, x_start = get_data(data)
arr[0,x_start] = 3

r = 0
while r<arr.shape[0]-1:
    dripped = False

    for c in range(arr.shape[1]):
        #Drip down
        if arr[r,c]==3 and arr[r+1,c]==0:
            arr = drip_down(arr, r,c)
        #Drip across - if we dripped across we also need to decrement our row index by 1
        if arr[r,c]==3 and arr[r+1,c] in [1,2]:
            arr, dripped_ = drip_across(arr, r,c)
            if dripped_:
                dripped = True
                
    if dripped:
        r-=1
    else:
        r+=1

sum_reservoir = np.sum(arr[1:,]==2)
sum_drips = np.sum(arr[1:,]==3)
print(sum_reservoir+sum_drips)

30384


In [10]:
print(sum_reservoir)

24479
